In [27]:


import torch as th
import torch.distributions as dist

import numpy as np

import networkx as nx

import pandas as pd

import matplotlib.pyplot as plt

class ProdGraph:
    def __init__(self, adjacency, distributions, ):
        self.adjacency = adjacency
        self.distributions = distributions
        
        G = nx.DiGraph(adjacency)
        topological_order = th.tensor(list(nx.topological_sort(G)))
        self.topological_order = topological_order
        
        
        self.remaining_time = th.zeros(adjacency.shape[0])
        self.states = th.zeros(adjacency.shape[0])
        self.output_buffer = th.zeros(adjacency.shape[0])
        self.n_nodes = adjacency.shape[0]
        
        
        # Initialize
        self.states[topological_order[0]] = 1
        self.lapsed_time = 0
        
        
        self.log = pd.DataFrame({'time': [], 'node': [], 'state': []})
        
    def forward(self):
        
        # Define Entry Point
        decision = th.where(self.states == 1, 1, 0)
        remaining_time = th.where(decision == 1, self.remaining_time, float('inf'))
        min_value, index = th.min(remaining_time, dim=0)
        
        # Iterate through Graph
        topological_index_entry_node = th.where(self.topological_order == index)[0].item()
        current_topological_order = [self.topological_order[(topological_index_entry_node + i) % self.n_nodes] for i in range(self.n_nodes)]
        
        for node_index in current_topological_order:
            self.output_buffer[index] += 1
        
        # Adjust times
        lapsed_time = min_value.item()
        self.remaining_time - lapsed_time
        self.lapsed_time += lapsed_time
        self.remaining_time[index] = self.distributions[index].sample()
        
        # Write to Log
        new_row_data = pd.DataFrame([{'time': self.lapsed_time, 'node': index.item(), 'state': 'Active'}])
        self.log = pd.concat([self.log, new_row_data])

        
        
        
        

In [28]:
adjacency_matrix = np.array([[0, 1, 0, 0],
                             [0, 0, 1, 0],
                             [0, 0, 0, 1],
                             [0, 0, 0, 0]])

dists = [dist.Uniform(9, 11), dist.Uniform(9, 11), dist.Uniform(9, 11), dist.Uniform(9, 11)]

graph = ProdGraph(adjacency_matrix, dists)

for i in range(50):
    graph.forward()
    
graph.log

,time,node,state
0,0.000000,0.0,Active
0,9.434729,0.0,Active
0,18.729580,0.0,Active
0,27.848368,0.0,Active
0,37.378354,0.0,Active
0,46.594888,0.0,Active
0,56.201851,0.0,Active
0,66.609346,0.0,Active
0,77.477155,0.0,Active
0,86.737842,0.0,Active


tensor(10.6433)